In [2]:
import json
from tqdm.notebook import tqdm

from transformers import AutoTokenizer, AutoModel
import os
from sentence_transformers.util import cos_sim
import pandas as pd

In [3]:
model_semantic_similarity = AutoModel.from_pretrained('/home/css/models/NV-Embed-v2', 
                                                      trust_remote_code=True, 
                                                      device_map="auto",
                                                      torch_dtype='bfloat16',
                                                      )

def calculate_semantic_similarity(sentence1, sentence2, max_length=32768):
    # 对输入的两个句子进行编码
    embeddings = model_semantic_similarity.encode([sentence1, sentence2], 
                                                  instruction="", 
                                                  max_length=max_length)
    
    # 计算余弦相似度
    similarity = cos_sim(embeddings[0], embeddings[1])
    
    return similarity.item()

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

We've detected an older driver with an RTX 4000 series GPU. These drivers have issues with P2P. This can affect the multi-gpu inference when using accelerate device_map.Please make sure to update your driver to the latest version which resolves this.


In [5]:
threshold = 0.9426

with open("dataset/jfleg.json", "r", encoding="utf-8") as f:
    data = json.load(f)

lst_original = data['original']
lst_wsc = data['wsc']

lst_rollback = []
for i, item in tqdm(enumerate(lst_original), total=len(lst_original)):
    ori_text = item
    cor_text = lst_wsc[i]
    if calculate_semantic_similarity(ori_text, cor_text) < threshold:
        lst_rollback.append(ori_text)
    else:
        lst_rollback.append(cor_text)

# 保存wsc后的数据
dic = {
    "original":lst_original,
    "wsc":lst_wsc,
    "rollback":lst_rollback
}
with open("dataset/jfleg.json", "w", encoding="utf-8") as f:
    json.dump(dic, f, ensure_ascii=False, indent=4)

# 单独保存下游任务需要的txt文件
mid = len(lst_rollback) // 2
rollback_1=lst_rollback[:mid]
rollback_2=lst_rollback[mid:]
with open("dataset/jfleg_txt/jfleg_rollback/sources.txt", "w") as file:
    for i, line in enumerate(rollback_1):
        if i < len(rollback_1) - 1:
            file.write(line + "\n")
        else:
            file.write(line)

with open("dataset/jfleg_txt/jfleg_rollback/corrections.txt", "w") as file:
    for i, line in enumerate(rollback_2):
        if i < len(rollback_2) - 1:
            file.write(line + "\n")
        else:
            file.write(line)

  0%|          | 0/3202 [00:00<?, ?it/s]

/home/liujunhui/.cache/huggingface/modules/transformers_modules/NV-Embed-v2/modeling_nvembed.py:349: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'input_ids': torch.tensor(batch_dict.get('input_ids').to(batch_dict.get('input_ids')).long()),
/home/liujunhui/miniconda3/envs/semantic_uncertainty/lib/python3.11/contextlib.py:105: FutureWarning: `torch.backends.cuda.sdp_kernel()` is deprecated. In the future, this context manager will be removed. Please see `torch.nn.attention.sdpa_kernel()` for the new context manager, with updated signature.
  self.gen = func(*args, **kwds)
